## Generate SCD

In [ ]:
#!/usr/bin/env python
# scd_preprocess_tim_64.py
# ------------------------------------------------------------
# 依赖：PyTorch ≥2.0、SciPy、NumPy、tqdm
# Usage : python scd_preprocess_tim_64.py
# ------------------------------------------------------------

import os, gc, math, time
import numpy as np
import torch
from tqdm import tqdm
from scipy.signal import decimate, chebwin

# ------------------------------------------------------------
# 0. 全局常量
# ------------------------------------------------------------
torch.set_grad_enabled(False)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️  running on {DEVICE}")

ROOT_TIM   = "CSPB.ML_dataset/CSPB_ML_2022_Data"   # 28 × 4000 个 .tim
OUT_BASE   = "cached_scd_tim"                      # 总输出根目录
# ---------------- 信号与窗参数 ----------------
N          = 512           # 输入长度
DECIMATE_Q = 64            # 32768 → 512
Np, L, P   = 64, 16, 32    # 帧长 64，移位 16，共 32 帧
NN         = (P - 1)*L + Np           # 560
# ---------------- 数据集分块 ------------------
N_BATCHES  = 28            # Batch_Dir_1 … Batch_Dir_28
N_PER_DIR  = 4000          # 每批 4000 个信号
DTYPE_OUT  = np.float32    # 保存精度
# ---------------- 窗函数 (Chebyshev-64, at=100 dB) ----------
#WIN64 = chebwin(64, at=100).astype(np.float32)   # (64,)
w64 = np.hamming(64)     # 或 np.window.hamming(64) (NumPy ≥1.25)
WIN64 = w64
# ------------------------------------------------------------
# 1. 工具函数
# ------------------------------------------------------------
def decimate_complex(x: np.ndarray, q: int = DECIMATE_Q) -> np.ndarray:
    """I、Q 分量独立 FIR + zero_phase 降采样，再合成复数"""
    re = decimate(np.real(x), q, ftype='fir', zero_phase=True)
    im = decimate(np.imag(x), q, ftype='fir', zero_phase=True)
    return re + 1j * im

def iq2_scd_matrix_torch(iq_512: torch.Tensor) -> np.ndarray:
    """
    输入：512 点复数 IQ（torch.complex64, on GPU）
    输出：|SCD|² 结果 (16, 64*64) = (16, 4096)  numpy.float32
    """
    # --------- zero-pad 到 NN = 560 ----------
    if iq_512.numel() < NN:
        pad = torch.zeros(NN - iq_512.numel(),
                          dtype=iq_512.dtype, device=iq_512.device)
        cfloat_signal = torch.cat([iq_512, pad])
    else:
        cfloat_signal = iq_512

    # --------- 切 32 帧，每帧 64 点 ----------
    X = torch.stack([cfloat_signal[k*L : k*L+Np] for k in range(P)], dim=1)  # (64,32)

    # --------- 加窗 ----------
    W = torch.tensor(WIN64, device=iq_512.device)[:, None]          # (64,1)
    XW = W * X                                                      # (64,32)

    # --------- 第一次 FFT ----------
    XF1 = torch.fft.fft(XW, dim=0)            # (64,32)
    XF1_shift = torch.cat([XF1[Np//2:], XF1[:Np//2]], dim=0)

    # --------- 下变频 ----------
    k = torch.arange(-Np//2, Np//2, device=iq_512.device)[:, None]  # (64,1)
    idx = torch.arange(P, device=iq_512.device)[None, :]            # (1,32)
    E = torch.exp(-1j * 2 * math.pi * k * idx * L / Np)             # (64,32)
    XD = (XF1_shift * E).T                                          # (32,64)

    # --------- 自互相关 & 第二次 FFT ----------
    XM  = XD[:, :, None] * XD[:, None, :].conj()    # (32,64,64)
    XM  = XM.reshape(P, -1)                         # (32,4096)
    XF2 = torch.fft.fft(XM, dim=0)                  # (32,4096)
    Z   = torch.abs(XF2) ** 2                       # (32,4096)

    # --------- 拼接 16 周期段 ----------
    out = torch.zeros((16, Z.shape[1]), device=iq_512.device)
    out[:8] = Z[P//2 : 3*P//4]      # 16…23
    out[8:] = Z[P//4 : P//2]        # 8…15
    return out.cpu().numpy().astype(np.float32)

def create_label():
    """返回 (4000,1) long 张量：0–7 重复 500 次"""
    base = torch.tensor([0,1,2,3,4,5,6,7], dtype=torch.long)
    y = base.repeat(500).reshape(-1,1)          # 8×500=4000
    return y

def tim_to_full64(tim_path: str) -> np.ndarray:
    """单个 .tim → 中心 64×64 SCD 热点矩阵 (float32)"""
    raw = np.fromfile(tim_path, dtype=np.single)
    iq_full = raw[2::2] + 1j * raw[3::2]          # 32768
    iq_512  = decimate_complex(iq_full)           # 512
    scd16   = iq2_scd_matrix_torch(
        torch.from_numpy(iq_512.astype(np.complex64)).to(DEVICE)
    )
    return scd16[8].reshape(64, 64).T             # (64,64)

# ------------------------------------------------------------
# 2. 单批处理并保存 .pt
# ------------------------------------------------------------
def process_one_dir(batch_idx: int):
    in_dir = os.path.join(ROOT_TIM, f"Batch_Dir_{batch_idx}")
    X = torch.empty((N_PER_DIR, 64, 64), dtype=torch.float32)

    for n in tqdm(range(1, N_PER_DIR + 1),
                  desc=f"Batch {batch_idx}", ncols=75):
        g = (batch_idx - 1) * N_PER_DIR + n
        full64 = tim_to_full64(os.path.join(in_dir, f"signal_{g}.tim"))
        X[n-1].copy_(torch.from_numpy(full64))
        torch.cuda.empty_cache()

    y = create_label()
    out_dir  = os.path.join(OUT_BASE, "2022_c_64_512sample_hamming")
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"2022_batch_{batch_idx:02d}.pt")
    torch.save((X, y), out_path)
    print(f"saved {out_path} | X {tuple(X.shape)}  y {tuple(y.shape)}")

    del X, y; gc.collect()

# ------------------------------------------------------------
# 3. 主入口
# ------------------------------------------------------------
if __name__ == "__main__":
    t0 = time.time()
    for idx in range(1, N_BATCHES + 1):
        process_one_dir(idx)
    print(f"\n🏁 all finished in {(time.time() - t0)/60:.2f} min")


⚙️  running on cuda


Batch 1: 100%|████████████████████████| 4000/4000 [00:32<00:00, 123.68it/s]


✅ saved cached_scd_tim/2022_c_64_512sample_hamming/2022_batch_01.pt | X (4000, 64, 64)  y (4000, 1)


Batch 2: 100%|████████████████████████| 4000/4000 [00:36<00:00, 109.00it/s]


✅ saved cached_scd_tim/2022_c_64_512sample_hamming/2022_batch_02.pt | X (4000, 64, 64)  y (4000, 1)


...

✅ saved cached_scd_tim/2022_c_64_512sample_hamming/2022_batch_26.pt | X (4000, 64, 64)  y (4000, 1)


Batch 27: 100%|███████████████████████| 4000/4000 [00:29<00:00, 134.45it/s]


✅ saved cached_scd_tim/2022_c_64_512sample_hamming/2022_batch_27.pt | X (4000, 64, 64)  y (4000, 1)


Batch 28: 100%|███████████████████████| 4000/4000 [00:31<00:00, 126.15it/s]


✅ saved cached_scd_tim/2022_c_64_512sample_hamming/2022_batch_28.pt | X (4000, 64, 64)  y (4000, 1)

🏁 all finished in 14.84 min
